In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt

In [3]:
from lobgpt.hdb import get_dataset

ModuleNotFoundError: No module named 'lobgpt'

In [ ]:
RCS = [f"{coin}USDT" for coin in ["BTC", "ETH", "XRP", "SOL", "DOGE"]]
TIMES = ['250912.000100', '250913.215000']

# <font color="grey">Data loading and handling</font>

Data is available in two forms: limit order book (LOB) and data sampled at fixed intervals (for example, seconds). The timestamps are given in nanosecond resolution as Unix timestamps, but the granularity of the data is not that fine. A simple dataloader and some helper function to convert Python datetime objects or strings of the form '240802.1450' into timestamps are provided. The 'book' data contains the bid and ask prices and volumes up to a specified depth (the maximum depth is currently 25).

In [ ]:
# At the moment, we are only concerned with data available through the Tardis dataset.
dl = get_dataset("tardis")

####  <a id='chapter1'> <font color="grey">1. Limit Order Book</font></a>

In [ ]:
df = dl.load_book("BTCUSDT", TIMES, depth=10)

In [ ]:
df.shape

In [ ]:
# Add human readable timestamp and mid prices
df = df.ds.add_datetime('ts')
df.head()

In [ ]:
plt.plot(df['dts'], df['asks[0].price'], df['dts'], df['bids[0].price'])
plt.xlabel('Time')
plt.ylabel('Bid and ask prices')
plt.title(f'{RCS[0]}')
plt.show()

In [ ]:
df = dl.load_book("BTCUSDT", ["250905.120000", "250905.130000"], depth=1)
df = df.ds.add_datetime('ts')
fig, ax = plt.subplots(1,2, figsize=(12,6))
ax[0].plot(df['dts'], df['asks[0].price'], df['dts'], df['bids[0].price'])
ax[1].plot(df['dts'], df['asks[0].price'] - df['bids[0].price'])
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Prices')
ax[1].set_xlabel('Time')
ax[1].set_ylabel('Bid-ask spread')
plt.show()

####  <a id='chapter2'> <font color="grey">2. Fixed-frequency data (bars)</font></a>

In [ ]:
df = dl.load_sync(["BTCUSDT", "ETHUSDT"], ["250605.120000", "250605.150000"], col="mid", freq="1s")

In [ ]:
df.head()

In [ ]:
plt.plot(df['ts'], df['mid_BTCUSDT'])
plt.show()

####  <a id='chapter3'> <font color="grey">3. Trades</font></a>

In [ ]:
tdf = dl.load_trades(["BTCUSDT", "ETHUSDT", "SOLUSDT", "XRPUSDT", "DOGEUSDT"], TIMES)
tdf.head()

####  <a id='chapter4'> <font color="grey">4. PyTorch Dataset</font></a>

In [ ]:
# Add TLOB labels
df = add_tlob_labels(df, horizons=[10, 20, 50])
df = add_lob_features(df, depth=10)

In [ ]:
df.head()